In [1]:
import pandas as pd
import requests
import json
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time
time.sleep(0.3)
from pykrx import stock
import dart_fss as dart
from dart_fss.errors import NotFoundConsolidated
pd.options.display.float_format = '{:.6f}'.format

In [2]:
cdf = pd.read_csv('data/company_name_and_code.csv', index_col = 0)
cdf

,corp_code,corp_name,stock_code
0,365387,AJ네트웍스,95570
1,125080,AK홀딩스,6840
2,219097,BGF,27410
3,1263022,BGF리테일,282330
4,858364,BNK금융지주,138930
...,...,...,...
726,111421,휴니드테크놀러지스,5870
727,362238,휴비스,79980
728,156488,휴스틸,5010
729,103176,흥국화재,540


In [3]:
my_key = pd.read_csv('data/dart_key.txt', header = None)[0][0]

In [4]:
def revenue(cdf, my_key, start_year=2019, end_year=2024):

    base_url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.xml'
    reprt_code = '11011'  # 사업보고서

    corp_map = cdf[['corp_code', 'corp_name']].copy()
    corp_map['corp_code'] = corp_map['corp_code'].astype(str).str.zfill(8)
    corp_codes = corp_map['corp_code']

    year_dfs = {}

    revenue_account = 'ifrs-full_Revenue'  # 매출액

    for year in range(start_year, end_year + 1):
        print(f"\n=== {year}년 진행 ===")
        all_dfs = []

        for idx, code in enumerate(
            tqdm(corp_codes, desc=f"{year}년 기업 진행", ncols=100),
            start=1
        ):

            def fetch_financial(fs_div):
                params = {
                    'crtfc_key': my_key,
                    'corp_code': code,
                    'bsns_year': str(year),
                    'reprt_code': reprt_code,
                    'fs_div': fs_div
                }

                try:
                    r = requests.get(base_url, params=params, timeout=10)
                    r.raise_for_status()
                except requests.RequestException:
                    return None, None

                soup = BeautifulSoup(r.text, "lxml-xml")

                status = soup.find('status')
                if status and status.text != '000':
                    return None, None

                lists = soup.find_all('list')

                current_rev = None
                previous_rev = None

                for item in lists:
                    account_id_tag = item.find('account_id')
                    if not account_id_tag:
                        continue
                    account_id = account_id_tag.text.strip()

                    if account_id == revenue_account:
                        thstrm = item.find('thstrm_amount')
                        frmtrm = item.find('frmtrm_amount')
                        current_rev = thstrm.text.strip() if thstrm else None
                        previous_rev = frmtrm.text.strip() if frmtrm else None
                        break

                return current_rev, previous_rev

            # CFS 조회
            curr_rev, prev_rev = fetch_financial('CFS')

            # 없으면 OFS에서 조회
            if curr_rev is None:
                curr_rev, prev_rev = fetch_financial('OFS')

            if curr_rev is not None:
                all_dfs.append(pd.DataFrame([{
                    'corp_code': code,
                    'year': year,
                    'previous_revenue': prev_rev,
                    'current_revenue': curr_rev
                }]))

            # 속도 제어
            time.sleep(0.3)
            if idx % 50 == 0:
                time.sleep(5)

        # 연도별 raw DF
        if all_dfs:
            year_raw = pd.concat(all_dfs, ignore_index=True)
        else:
            year_raw = pd.DataFrame(columns=['corp_code', 'year', 'previous_revenue', 'current_revenue'])

        base_df = corp_map.copy()
        base_df['year'] = year

        year_df = base_df.merge(
            year_raw,
            on=['corp_code', 'year'],
            how='left'
        )

        # 숫자형 변환 및 결측값 처리
        year_df['previous_revenue'] = pd.to_numeric(year_df['previous_revenue'], errors='coerce').fillna(0)
        year_df['current_revenue'] = pd.to_numeric(year_df['current_revenue'], errors='coerce').fillna(0)

        year_df = year_df[['corp_code', 'corp_name', 'year', 'previous_revenue', 'current_revenue']]

        year_dfs[year] = year_df

    # 튜플로 반환 (2019~2024)
    return tuple(year_dfs[year] for year in range(start_year, end_year + 1))

In [5]:
adf_2019, adf_2020, adf_2021, adf_2022, adf_2023, adf_2024 = revenue(cdf, my_key, 2019, 2024)


=== 2019년 진행 ===


2019년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [08:04<00:00,  1.51it/s]



=== 2020년 진행 ===


2020년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [07:58<00:00,  1.53it/s]



=== 2021년 진행 ===


...

2022년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [07:52<00:00,  1.55it/s]



=== 2023년 진행 ===


2023년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [08:11<00:00,  1.49it/s]



=== 2024년 진행 ===


2024년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [08:08<00:00,  1.50it/s]


In [6]:
adf_lst = [adf_2019, adf_2020, adf_2021, adf_2022, adf_2023, adf_2024]

for i in adf_lst:
    print(len(i[(i['previous_revenue'] == 0) | (i['current_revenue'] == 0)]))

61
52
51
52
31
28


In [7]:
adf_lst = [
    adf_2019, adf_2020, adf_2021, adf_2022, adf_2023, adf_2024
]

In [8]:
for df, year in zip(adf_lst, range(2019, 2025)):
    df.to_csv(
        f'data/revenue/revenue_{year}.csv',
        float_format='%.6f',
        index=False
    )

In [9]:
adf_2018 = revenue(cdf, my_key, 2018, 2018)


=== 2018년 진행 ===


2018년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [09:46<00:00,  1.25it/s]


In [10]:
adf_2018 = adf_2018[0]
adf_2018

,corp_code,corp_name,year,previous_revenue,current_revenue
0,00365387,AJ네트웍스,2018,0.000000,0.000000
1,00125080,AK홀딩스,2018,0.000000,0.000000
2,00219097,BGF,2018,0.000000,0.000000
3,01263022,BGF리테일,2018,0.000000,0.000000
4,00858364,BNK금융지주,2018,0.000000,0.000000
...,...,...,...,...,...
726,00111421,휴니드테크놀러지스,2018,0.000000,0.000000
727,00362238,휴비스,2018,0.000000,0.000000
728,00156488,휴스틸,2018,0.000000,0.000000
729,00103176,흥국화재,2018,0.000000,0.000000


## 데이터 전처리

In [11]:
adf_2019 = pd.read_csv('data/revenue/revenue_2019.csv')
adf_2020 = pd.read_csv('data/revenue/revenue_2020.csv')
adf_2021 = pd.read_csv('data/revenue/revenue_2021.csv')
adf_2022 = pd.read_csv('data/revenue/revenue_2022.csv')
adf_2023 = pd.read_csv('data/revenue/revenue_2023.csv')
adf_2024 = pd.read_csv('data/revenue/revenue_2024.csv')

In [12]:
all_19 = pd.read_csv('data/data_clean/2019(연간)/2019_연간보고서_포괄손익계산서전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '매출액']]
all_20 = pd.read_csv('data/data_clean/2020연간/2020_연간보고서_포괄손익계산서전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '매출액']]
all_21 = pd.read_csv('data/data_clean/2021연간/2021_연간보고서_포괄손익계산서전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '매출액']]
all_22 = pd.read_csv('data/data_clean/2022연간/2022_연간보고서_포괄손익계산서전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '매출액']]
all_23 = pd.read_csv('data/data_clean/2023연간/2023_연간보고서_포괄손익계산서전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '매출액']]
all_24 = pd.read_csv('data/data_clean/2024연간/2024_연간보고서_포괄손익계산서전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '매출액']]

In [13]:
new_19 = pd.read_csv('data/data_clean/2019(연간)/2019_연간보고서_손익계산서전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '매출액']]
new_20 = pd.read_csv('data/data_clean/2020연간/2020_연간보고서_손익계산서전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '매출액']]
new_21 = pd.read_csv('data/data_clean/2021연간/2021_연간보고서_손익계산서전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '매출액']]
new_22 = pd.read_csv('data/data_clean/2022연간/2022_연간보고서_손익계산서전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '매출액']]
new_23 = pd.read_csv('data/data_clean/2023연간/2023_연간보고서_손익계산서전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '매출액']]
new_24 = pd.read_csv('data/data_clean/2024연간/2024_연간보고서_손익계산서전처리.txt', sep = '\t', encoding ='cp949')[['회사명', '매출액']]

In [14]:
ca_19 = adf_2019[(adf_2019['previous_revenue'] == 0) | (adf_2019['current_revenue'] == 0)]
ca_20 = adf_2020[(adf_2020['previous_revenue'] == 0) | (adf_2020['current_revenue'] == 0)]
ca_21 = adf_2021[(adf_2021['previous_revenue'] == 0) | (adf_2021['current_revenue'] == 0)]
ca_22 = adf_2022[(adf_2022['previous_revenue'] == 0) | (adf_2022['current_revenue'] == 0)]
ca_23 = adf_2023[(adf_2023['previous_revenue'] == 0) | (adf_2023['current_revenue'] == 0)]
ca_24 = adf_2024[(adf_2024['previous_revenue'] == 0) | (adf_2024['current_revenue'] == 0)]

In [15]:
print(
len(ca_19), len(ca_20), len(ca_21), len(ca_22), len(ca_23), len(ca_24)
)

61 52 51 52 31 28


In [16]:
adf_columns = ['corp_name', 'current_revenue']
all_19.columns = adf_columns
all_20.columns = adf_columns
all_21.columns = adf_columns
all_22.columns = adf_columns
all_23.columns = adf_columns
all_24.columns = adf_columns
all_19

,corp_name,current_revenue
0,3S,11086501016.000000
1,AJ네트웍스,300085588553.000000
2,AK홀딩스,20935560773.000000
3,APS,NaN
4,AP시스템,219323441981.000000
...,...,...
2685,흥국,51790956507.000000
2686,흥국에프엔비,NaN
2687,흥아해운,77832743695.000000
2688,희림,113948352190.000000


In [17]:
adf_columns = ['corp_name', 'current_revenue']
new_19.columns = adf_columns
new_20.columns = adf_columns
new_21.columns = adf_columns
new_22.columns = adf_columns
new_23.columns = adf_columns
new_24.columns = adf_columns
new_19

,corp_name,current_revenue
0,CJ,181742606000.000000
1,CJ대한통운,6596274009485.000000
2,DB하이텍,807416422918.000000
3,DSR,NaN
4,GS,NaN
...,...,...
193,호텔신라,NaN
194,화천기계,159658282529.000000
195,효성티앤씨,3739477556452.000000
196,효성화학,1813137241909.000000


In [18]:
cadf_2018 = adf_2018.copy()
cadf_2019 = adf_2019.copy()
cadf_2020 = adf_2020.copy()
cadf_2021 = adf_2021.copy()
cadf_2022 = adf_2022.copy()
cadf_2023 = adf_2023.copy()
cadf_2024 = adf_2024.copy()

In [19]:
cadf_lst = [cadf_2019, cadf_2020, cadf_2021, cadf_2022, cadf_2023, cadf_2024]
all_lst = [all_19, all_20, all_21, all_22, all_23, all_24]
new_lst = [new_19, new_20, new_21, new_22, new_23, new_24]

for cdf, adf, ndf in zip(cadf_lst, all_lst, new_lst):
    cdf = cdf.merge(
    adf[['corp_name', 'current_revenue']],
    on='corp_name',
    how='left')
    
    cdf = cdf.merge(
    ndf[['corp_name', 'current_revenue']],
    on='corp_name',
    how='left')
    
    mask_a = cdf['current_revenue_y'].notnull()
    mask_n = (cdf['current_revenue_x'] == 0) & (cdf['current_revenue'].notnull())
    
    cdf.loc[mask_a, 'current_revenue_x'] = cdf.loc[mask_a, 'current_revenue_y']
    cdf.loc[mask_n, 'current_revenue_x'] = cdf.loc[mask_n, 'current_revenue']
    
    cdf = cdf.drop(['current_revenue_y', 'current_revenue'], axis = 1)
    cdf.columns = ['corp_code', 'corp_name', 'year', 'previous_revenue', 'current_revenue']
    
    print(len(cdf[(cdf['previous_revenue'] == 0) | (cdf['current_revenue'] == 0)]))

61
52
51
52
31
28


In [20]:
n2019 = cadf_2019[(cadf_2019['previous_revenue'] == 0) | (cadf_2019['current_revenue'] == 0)]
n2020 = cadf_2020[(cadf_2020['previous_revenue'] == 0) | (cadf_2020['current_revenue'] == 0)]
n2021 = cadf_2021[(cadf_2021['previous_revenue'] == 0) | (cadf_2021['current_revenue'] == 0)]
n2022 = cadf_2022[(cadf_2022['previous_revenue'] == 0) | (cadf_2022['current_revenue'] == 0)]
n2023 = cadf_2023[(cadf_2023['previous_revenue'] == 0) | (cadf_2023['current_revenue'] == 0)]
n2024 = cadf_2024[(cadf_2024['previous_revenue'] == 0) | (cadf_2024['current_revenue'] == 0)]

In [21]:
print(
len(n2019), len(n2020), len(n2021), len(n2022), len(n2023), len(n2024))

61 52 51 52 31 28


In [22]:
cadf_2019[(cadf_2019['previous_revenue'] == 0) & (cadf_2019['current_revenue'] == 0)]

,corp_code,corp_name,year,previous_revenue,current_revenue
4,858364,BNK금융지주,2019,0.000000,0.000000
13,159102,DB손해보험,2019,0.000000,0.000000
14,115694,DB증권,2019,0.000000,0.000000
51,980122,JB금융지주,2019,0.000000,0.000000
56,688996,KB금융,2019,0.000000,0.000000
...,...,...,...,...,...
685,148610,한화투자증권,2019,0.000000,0.000000
702,137997,현대차증권,2019,0.000000,0.000000
705,164973,현대해상,2019,0.000000,0.000000
707,441243,형지엘리트,2019,0.000000,0.000000


In [23]:
# 이전 연도 리스트 (t)
prev_lst = [
    cadf_2018,
    cadf_2019,
    cadf_2020,
    cadf_2021,
    cadf_2022,
    cadf_2023
]

# 다음 연도 리스트 (t+1)
next_lst = [
    cadf_2019,
    cadf_2020,
    cadf_2021,
    cadf_2022,
    cadf_2023,
    cadf_2024
]

# 🔥 이전 → 다음 단방향 보완
for prev_df, next_df in zip(prev_lst, next_lst):

    mask = (
        (next_df['previous_revenue'] == 0) &
        (prev_df['current_revenue'] != 0)
    )

    next_df.loc[mask, 'previous_revenue'] = \
        prev_df.loc[mask, 'current_revenue']


In [24]:
cadf_2020.loc[
    (cadf_2020['previous_revenue'] == 0) &
    (cadf_2020['current_revenue'] != 0),
    ['corp_name', 'previous_revenue', 'current_revenue']
]


,corp_name,previous_revenue,current_revenue


In [25]:
cadf_2020.loc[
    cadf_2020['previous_revenue'] < 1,
    'previous_revenue'
].describe()


count   50.000000
mean     0.000000
std      0.000000
min      0.000000
25%      0.000000
50%      0.000000
75%      0.000000
max      0.000000
Name: previous_revenue, dtype: float64

In [26]:
cadf_2020[['previous_revenue', 'current_revenue']].dtypes


previous_revenue    float64
current_revenue     float64
dtype: object

In [27]:
cadf_2020.loc[
    cadf_2020['previous_revenue'] == 0,
    ['corp_name', 'previous_revenue', 'current_revenue']
]


,corp_name,previous_revenue,current_revenue
4,BNK금융지주,0.000000,0.000000
13,DB손해보험,0.000000,0.000000
14,DB증권,0.000000,0.000000
51,JB금융지주,0.000000,0.000000
56,KB금융,0.000000,0.000000
89,NH투자증권,0.000000,0.000000
90,NICE,0.000000,0.000000
114,SK증권,0.000000,0.000000
131,iM금융지주,0.000000,0.000000
153,교보증권,0.000000,0.000000


In [28]:
cadf_2019[cadf_2019['corp_name'] == 'SGC에너지']
cadf_2020[cadf_2020['corp_name'] == 'SGC에너지']

,corp_code,corp_name,year,previous_revenue,current_revenue
100,125150,SGC에너지,2020,284244834792.000000,106548366582.000000


In [29]:
cadf_2018

,corp_code,corp_name,year,previous_revenue,current_revenue
0,00365387,AJ네트웍스,2018,0.000000,0.000000
1,00125080,AK홀딩스,2018,0.000000,0.000000
2,00219097,BGF,2018,0.000000,0.000000
3,01263022,BGF리테일,2018,0.000000,0.000000
4,00858364,BNK금융지주,2018,0.000000,0.000000
...,...,...,...,...,...
726,00111421,휴니드테크놀러지스,2018,0.000000,0.000000
727,00362238,휴비스,2018,0.000000,0.000000
728,00156488,휴스틸,2018,0.000000,0.000000
729,00103176,흥국화재,2018,0.000000,0.000000


In [30]:
cadf_lst = [cadf_2019, cadf_2020, cadf_2021, cadf_2022, cadf_2023, cadf_2024]

for df, year in zip(cadf_lst, range(2019, 2025)):
    df.to_csv(
        f'data/revenue/revenue_{year}.csv',
        float_format='%.6f',
        index=False
    )